# MLP

In [1]:
import numpy as np
import time
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score


In [2]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1, f_activacion = "s"):
        self.f_activacion = f_activacion
        self.tiempo_entrenamiento = 0.0
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)
    
    def activacion_tanh(self, x):
        # Función de activación tanh (tangente hiperbólica)
        return np.tanh(x)

    def activacion_derivada_tanh(self, x):
        # Derivada de la función de activación tanh (tangente hiperbólica)
        return 1 - np.tanh(x)**2
    
    def activacion_relu(self, x):
        # Función de activación ReLU (Rectified Linear Unit)
        return np.maximum(0, x)

    def activacion_derivada_relu(self, x):
        # Derivada de la función de activación ReLU (Rectified Linear Unit)
        return np.where(x <= 0, 0, 1)
    
    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]

            if self.f_activacion == "r":
                y = self.activacion_relu(x)
            elif self.f_activacion == "t":
                y = self.activacion_tanh(x)
            else:
                y = self.activacion(x)

            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y

        if self.f_activacion == "r":
            delta = error * self.activacion_derivada_relu(capa_activacion[-1])
        elif self.f_activacion == "t":
            delta = error * self.activacion_derivada_tanh(capa_activacion[-1])
        else:
            delta = error * self.activacion_derivada(capa_activacion[-1])

        
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias

            if self.f_activacion == "r":
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada_relu(activacion_actual)
            elif self.f_activacion == "t":
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada_tanh(activacion_actual)
            else:
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def entrenar(self, X, y, epochs):
        tiempo_inicio = time.time()
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)
        self.tiempo_entrenamiento = time.time() - tiempo_inicio

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    
    def metricas(self, y_prueba, y_pred):
        recall = recall_score(y_prueba, y_pred, average='macro')
        precision = precision_score(y_prueba, y_pred, zero_division=1, average='macro')
        accuracy = accuracy_score(y_prueba, y_pred)
        f1 = f1_score(y_prueba, y_pred, average='macro')
        return recall, precision, accuracy, f1, self.tiempo_entrenamiento
    
    def imprimir_metricas(self, y_prueba, y_pred):
        recall, precision, accuracy, f1, tiempo_entrenamiento = self.metricas(y_prueba, y_pred)
        print ("Recall: ", recall)
        print ("Precision: ", precision)
        print ("Accuracy: ", accuracy)
        print ("F1: ", f1)
        print ("Tiempo de entrenamiento: ", tiempo_entrenamiento)



    

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
# Cargar el conjunto de datos Iris

iris = load_iris()
X = iris.data
y = iris.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [4]:
mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

In [5]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.225)
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=1000)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

predicciones = np.array(predicciones)
print ("Predicciones___: ", predicciones)
print ("Prueba_________: ", y_prueba)
perceptron.imprimir_metricas(y_prueba, predicciones)

Predicciones___:  [0 1 1 1 0 1 0 0 1 1 2 1 2 1 0 0 0 1 1 1 0 2 1 2 2 1 1 0 1 0]
Prueba_________:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Recall:  0.8333333333333334
Precision:  0.8888888888888888
Accuracy:  0.8333333333333334
F1:  0.8222222222222223
Tiempo de entrenamiento:  8.198851823806763


**3.1** Ejecute el código con un α = 0,15 y con epochs = 3000

In [14]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15)
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

predicciones = np.array(predicciones)
print ("Predicciones___: ", predicciones)
print ("Prueba_________: ", y_prueba)
perceptron.imprimir_metricas(y_prueba, predicciones)

Predicciones___:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Prueba_________:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Recall:  1.0
Precision:  1.0
Accuracy:  1.0
F1:  1.0
Tiempo de entrenamiento:  24.678574323654175


**3.2** Implemente la función de activación tanh y ReLU. Para cada una de ellas ejecute el modelo
y obtenga las métricas.

Nota: la implementación de estas dos funciones de activación se pueden observar en el bloque de código donde está definida la clase. Se necesita definir el atributo f_activación como "s" para la función sigmoide, "t" para la función tangente hiperbólica y como "r" para la función ReLU. Esto se puede observar de mejor manera en las próximas ejecuciones.

**3.2.1** Ejecución y métricas de *tanh*

In [16]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15, f_activacion="t")
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
print(perceptron.f_activacion)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

predicciones = np.array(predicciones)
print ("Predicciones___: ", predicciones)
print ("Prueba_________: ", y_prueba)
perceptron.imprimir_metricas(y_prueba, predicciones)

t
Predicciones___:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Prueba_________:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Recall:  0.3333333333333333
Precision:  0.7777777777777777
Accuracy:  0.3333333333333333
F1:  0.16666666666666666
Tiempo de entrenamiento:  24.631563425064087


**3.2.2** Ejecución y métricas de *ReLU*

In [8]:
# Crear y entrenar el perceptrón multicapa
perceptron_ReLU = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15, f_activacion="r") # "r" para usar la función de activación ReLU
perceptron_ReLU.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)

# Hacer predicciones sobre el conjunto de prueba
predicciones_ReLU = []
for i in range(len(X_prueba)):
    prediccion_ReLU = perceptron_ReLU.predecir(X_prueba[i])
    prediccion_clase_ReLU = np.argmax(prediccion_ReLU)
    predicciones_ReLU.append(prediccion_clase_ReLU)

predicciones_ReLU = np.array(predicciones_ReLU)
print ("Predicciones___: ", predicciones)
print ("Prueba_________: ", y_prueba)
perceptron_ReLU.imprimir_metricas(y_prueba, predicciones_ReLU)

Predicciones___:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Prueba_________:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Recall:  0.3333333333333333
Precision:  0.7777777777777777
Accuracy:  0.3333333333333333
F1:  0.16666666666666666
Tiempo de entrenamiento:  25.491756916046143


**3.3**  Ajuste los parámetros de epochs y α para mejorar los valores de las métricas.

**3.3.1** Ajuste para *sigmoide*

In [43]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.01) # Cambio de alpha: 0.15 a 
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=2000)

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

predicciones = np.array(predicciones)
perceptron.imprimir_metricas(y_prueba, predicciones)

Recall:  1.0
Precision:  1.0
Accuracy:  1.0
F1:  1.0
Tiempo de entrenamiento:  16.441713094711304


**3.3.1** Ajuste para *tanh*

In [38]:
# Crear y entrenar el perceptrón multicapa
perceptron_tanh = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.001, f_activacion="t") # "t" para usar la función de activación tanh
perceptron_tanh.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=1000)

# Hacer predicciones sobre el conjunto de prueba
predicciones_tanh = []
for i in range(len(X_prueba)):
    prediccion_tanh = perceptron_tanh.predecir(X_prueba[i])
    prediccion_clase_tanh = np.argmax(prediccion_tanh)
    predicciones_tanh.append(prediccion_clase_tanh)

predicciones_tanh = np.array(predicciones_tanh)
print (y_prueba)
print (predicciones_tanh)
perceptron_tanh.imprimir_metricas(y_prueba, predicciones_tanh)

[0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
[0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 2 2 0 2 1 2 2 2 1 0 2 0]
Recall:  0.9333333333333332
Precision:  0.9444444444444445
Accuracy:  0.9333333333333333
F1:  0.9326599326599326
Tiempo de entrenamiento:  8.213855504989624


**3.3.1** Ajuste para *ReLU*

In [41]:
# Crear y entrenar el perceptrón multicapa
perceptron_ReLU = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.0001, f_activacion="r") # "r" para usar la función de activación ReLU
perceptron_ReLU.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=1000)

# Hacer predicciones sobre el conjunto de prueba
predicciones_ReLU = []
for i in range(len(X_prueba)):
    prediccion_ReLU = perceptron_ReLU.predecir(X_prueba[i])
    prediccion_clase_ReLU = np.argmax(prediccion_ReLU)
    predicciones_ReLU.append(prediccion_clase_ReLU)

predicciones_ReLU = np.array(predicciones_ReLU)
perceptron_ReLU.imprimir_metricas(y_prueba, predicciones_ReLU)

Recall:  0.8666666666666667
Precision:  0.9047619047619048
Accuracy:  0.8666666666666667
F1:  0.861111111111111
Tiempo de entrenamiento:  8.576937198638916


**3.4** Determine cuál función de activación arrojó mejores resultados

En este caso intentando ajustar los hiperparámetros *alpha* y *epochs* para cada función de activación, se obtiene que el mejor resultado fue obtenido por la función *sigmoide*.

**3.5** Una vez que se haya obtenido los resultados anteriores obtenga el mejor resultado utilizando
la biblioteca sklearn. Se sugiere utilizar Gridsearch para encontrar los parámetros.

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Ajustando clase para utilizar GridSearch
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1, epochs=0.1, f_activacion = "s"):
        self.f_activacion = f_activacion
        self.tiempo_entrenamiento = 0.0
        self.capas = capas
        self.alpha = alpha
        self.epochs = epochs
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def get_params(self, deep=True):
        # Devuelve un diccionario con los parámetros del objeto
        return {
            'capas': self.capas,
            'alpha': self.alpha,
            'epochs': self.epochs,
            'f_activacion': self.f_activacion
        }
    
    def set_params(self, **params):
        # Actualiza los parámetros del objeto con los valores proporcionados en el diccionario params
        if 'capas' in params:
            self.capas = params['capas']
        if 'alpha' in params:
            self.alpha = params['alpha']
        if 'epochs' in params:
            self.epochs = params['epochs']
        if 'f_activacion' in params:
            self.f_activacion = params['f_activacion']
        
        return self

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)
    
    def activacion_tanh(self, x):
        # Función de activación tanh (tangente hiperbólica)
        return np.tanh(x)

    def activacion_derivada_tanh(self, x):
        # Derivada de la función de activación tanh (tangente hiperbólica)
        return 1 - np.tanh(x)**2
    
    def activacion_relu(self, x):
        # Función de activación ReLU (Rectified Linear Unit)
        return np.maximum(0, x)

    def activacion_derivada_relu(self, x):
        # Derivada de la función de activación ReLU (Rectified Linear Unit)
        return np.where(x <= 0, 0, 1)
    
    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]

            if self.f_activacion == "r":
                y = self.activacion_relu(x)
            elif self.f_activacion == "t":
                y = self.activacion_tanh(x)
            else:
                y = self.activacion(x)

            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y

        if self.f_activacion == "r":
            delta = error * self.activacion_derivada_relu(capa_activacion[-1])
        elif self.f_activacion == "t":
            delta = error * self.activacion_derivada_tanh(capa_activacion[-1])
        else:
            delta = error * self.activacion_derivada(capa_activacion[-1])

        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias

            if self.f_activacion == "r":
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada_relu(activacion_actual)
            elif self.f_activacion == "t":
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada_tanh(activacion_actual)
            else:
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def fit(self, X, y):
        tiempo_inicio = time.time()
        for epoch in range(0, self.epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)
        self.tiempo_entrenamiento = time.time() - tiempo_inicio

    def predict(self, X):

        # Hacer predicciones sobre el conjunto de prueba
        predicciones = []
        for i in range(len(X)):
            # Obtener la salida de la última capa
            capa_activacion = self.feedforward(X[i])
            
            prediccion = capa_activacion[-1]
            prediccion_clase = np.argmax(prediccion)
            predicciones.append(prediccion_clase)

        predicciones = np.array(predicciones)
        return predicciones
    
    def score(self, X, y):
        y_pred = self.predict(X)
        accuracy = accuracy_score(y, y_pred)
        return accuracy
    
    def metricas(self, y_prueba, y_pred):
        recall = recall_score(y_prueba, y_pred, average='macro')
        precision = precision_score(y_prueba, y_pred, zero_division=1, average='macro')
        accuracy = accuracy_score(y_prueba, y_pred)
        f1 = f1_score(y_prueba, y_pred, average='macro')
        return recall, precision, accuracy, f1, self.tiempo_entrenamiento
    
    def imprimir_metricas(self, y_prueba, y_pred):
        recall, precision, accuracy, f1, tiempo_entrenamiento = self.metricas(y_prueba, y_pred)
        print ("Recall: ", recall)
        print ("Precision: ", precision)
        print ("Accuracy: ", accuracy)
        print ("F1: ", f1)
        print ("Tiempo de entrenamiento: ", tiempo_entrenamiento)

**3.5.1** GridSearch para función de activación *sigmoide*

In [59]:
# Convertir las etiquetas a formato categórico
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_entrenamiento)

# Define los parámetros que deseas explorar en la búsqueda
param_grid = {
    'alpha': [0.0001],
    'epochs': [1000]}

# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], f_activacion="s")

# Crea una instancia de GridSearchCV con tu clase PerceptronMulticapa y los parámetros definidos
grid_search = GridSearchCV(perceptron, param_grid, cv=5)

# Realiza la búsqueda de hiperparámetros utilizando los datos de entrenamiento X_train y y_train
grid_search.fit(X_entrenamiento, y_encoded)

# Obtiene los mejores hiperparámetros encontrados
mejor_alpha = grid_search.best_params_['alpha']
mejor_epochs = grid_search.best_params_['epochs']

print (mejor_alpha)
print (mejor_epochs)


0.0001
1000


# MLP con Sklearn

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [13]:
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation= 'tanh', max_iter=10000, random_state=42, solver="sgd")
mlp.fit(X_entrenamiento, y_entrenamiento)

y_pred = mlp.predict(X_prueba)

print("Predicciones: ", y_pred)
print("Valor real  : ",y_prueba)

accuracy = accuracy_score(y_prueba, y_pred)
print("Precisión: {:.2f}".format(accuracy))

confusion_matrix(y_prueba,y_pred)

Predicciones:  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Valor real  :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.00


array([[10,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 10]], dtype=int64)